In [ ]:
!pip install -U spacy -q

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
nlp = spacy.blank("en")
db = DocBin()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
f = open('/content/drive/MyDrive/Colab Notebooks/MTP after 15/JUDGEMENT_EXTRACTION/annotations_judgement.json')
J_DATA = json.load(f)

In [ ]:
J_DATA

{'classes': ['COSTS',
  'DISPOSED OF',
  'APPEAL IS ALLOWED',
  'CANNOT BE SUSTAINED',
  'DISMISSED',
  'APPEAL IS DISMISSED'],
 'annotations': [['The petition is disposed of in the above terms. All pending applications are also disposed of.\r\nParties to bear their own costs\r\n',
   {'entities': [[16, 27, 'DISPOSED OF'],
     [82, 94, 'DISPOSED OF'],
     [122, 127, 'COSTS']]}],
  ['\r\n\r\nIn view of the foregoing discussion, we set aside the judgment and order dated 12.01.2015\r\npassed by the High Court. The appeal is allowed\r\n',
   {'entities': [[125, 142, 'APPEAL IS ALLOWED']]}],
  ['\r\n\r\nThe impugned award cannot be sustained and the same is set aside. The disputes are remitted to the\r\nTribunal for fresh adjudication in accordance with law on basis of cogent evidence including\r\ncontemporaneous records.Certified copy of this order, if applied for, be supplied to the parties subject to compliance of all\r\nrequisite formalities\r\n',
   {'entities': [[23, 42, 'CANNOT BE 

In [ ]:
for text, annot in tqdm(J_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
           print("Skipping entity")
        else:
           ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data_precedent_02_Judgement.spacy")

100%|██████████| 32/32 [00:00<00:00, 1743.04it/s]


In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

2022-10-05 14:00:16.126612: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data_precedent_02_Judgement.spacy --paths.dev ./training_data_precedent_02_Judgement.spacy

2022-10-05 14:00:50.252859: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-10-05 14:00:51,022] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-10-05 14:00:51,036] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-10-05 14:00:51,042] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-10-05 14:00:51,043] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-10-05 14:00:51,322] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #  

In [ ]:
nlp_ner = spacy.load("/content/model-last")

In [ ]:
import pickle

In [ ]:
with open('model_precedent2_pickle_Judgement', 'wb') as f:
  pickle.dump(nlp_ner, f)

In [ ]:
with open("/content/jtest01.txt") as f1:
  x = f1.read()

In [ ]:
doc1 = nlp_ner(x)

In [ ]:
spacy.displacy.render(doc1, style="ent", jupyter=True)